### PySpark Data Processing Project Checklist

#### 1. Data Loading
- [ ] Load `orders.csv` into a Spark DataFrame
- [ ] Load `customers.csv` into a Spark DataFrame

#### 2. Data Transformation
- [ ] Merge the datasets on `customer_id`
- [ ] Make all columns uppercase (optional)
- [ ] Calculate each customer's total spend across all orders
- [ ] Create a new column `days_since_first_order` indicating the number of days since the customer first ordered

#### 3. Aggregation
- [ ] Group data by `product_category`
- [ ] Calculate the total order amount for each category
- [ ] Identify the top 3 customers based on total spend

#### 4. Data Export
- [ ] Create `customer_spend.csv` with columns:
  - [ ] `customer_id`
  - [ ] `customer_name`
  - [ ] `total_spend`
  - [ ] `days_since_first_order`
- [ ] Create `category_summary.csv` with columns:
  - [ ] `product_category`
  - [ ] `total_order_amount`
- [ ] Verify both CSV files were saved correctly

#### 5. Validation (Optional)
- [ ] Verify row counts match expectations
- [ ] Check for null values in critical columns
- [ ] Validate calculations with sample data

In [7]:
# create spark session 
import os 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, sum, avg, count, row_number, round, dayofmonth, min, max, current_date, datediff, upper, to_date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
from pyspark.sql.window import Window

spark = SparkSession.builder \
        .appName("ecomm_orders_etl") \
        .master("local[*]") \
        .getOrCreate()

In [11]:
# load data 
orders_df = spark.read.csv("source_data/orders.csv", header=True, inferSchema=True)
customers_df = spark.read.csv("source_data/customers.csv", header=True, inferSchema=True)

#validate 
orders_df.show(5)
customers_df.show(5)

+--------+-----------+----------+------------+----------------+
|order_id|customer_id|order_date|order_amount|product_category|
+--------+-----------+----------+------------+----------------+
|       1|        101|2025-01-10|         250|     Electronics|
|       2|        102|2025-02-15|         180|         Fashion|
|       3|        103|2025-02-17|          70|       Groceries|
|       4|        101|2025-03-01|         300|     Electronics|
|       5|        104|2025-03-12|         120|          Health|
+--------+-----------+----------+------------+----------------+

+-----------+-------------+--------------------+--------------+
|customer_id|customer_name|      customer_email|customer_since|
+-----------+-------------+--------------------+--------------+
|        101|     John Doe| johndoe@example.com|    2024-07-10|
|        102|   Jane Smith|janesmith@example...|    2023-12-05|
|        103|  Emily Davis|emilydavis@exampl...|    2025-01-20|
|        104|Michael Brown|michaelbrown

In [16]:
# transformations 
# 2.1 Merge the datasets on customer_id
joined_df = orders_df.join(customers_df, "customer_id")

# 2.2 Make all columns uppercase (optional)

# 2.3 Calculate each customer's total spend across all orders
# 2.4 Create a new column days_since_first_order indicating the number of days since the customer first ordered
window_spec = Window.partitionBy("customer_id")
t2DF = joined_df.withColumn("total_spend", sum(col("order_amount")).over(window_spec)) \
            .withColumn("first_order_date", min(col("order_date")).over(window_spec)) \
    .withColumn("days_since_first_order", datediff(current_date(), (col("order_date"))))

t2DF.show(5)

+-----------+--------+----------+------------+----------------+-------------+--------------------+--------------+-----------+----------------+----------------------+
|customer_id|order_id|order_date|order_amount|product_category|customer_name|      customer_email|customer_since|total_spend|first_order_date|days_since_first_order|
+-----------+--------+----------+------------+----------------+-------------+--------------------+--------------+-----------+----------------+----------------------+
|        101|       4|2025-03-01|         300|     Electronics|     John Doe| johndoe@example.com|    2024-07-10|        550|      2025-01-10|                   310|
|        101|       1|2025-01-10|         250|     Electronics|     John Doe| johndoe@example.com|    2024-07-10|        550|      2025-01-10|                   360|
|        102|       2|2025-02-15|         180|         Fashion|   Jane Smith|janesmith@example...|    2023-12-05|        180|      2025-02-15|                   324|
|   

In [18]:
# aggreation 
# 3.1 Group data by product_category
# 3.2 Calculate the total order amount for each category
# 3.3 Identify the top 3 customers based on total spend
from pyspark.sql.functions import dense_rank
t3DF = t2DF.groupBy("product_category", "customer_id").agg(sum("order_amount").alias("total_order_amount"))

window_spec_ = Window.partitionBy("customer_id").orderBy(col("total_order_amount").desc())
t4DF = t3DF.withColumn("ranked_spend", dense_rank().over(window_spec_)) \
            .filter(col("ranked_spend") <= 3)
t4DF.show(5)

+----------------+-----------+------------------+------------+
|product_category|customer_id|total_order_amount|ranked_spend|
+----------------+-----------+------------------+------------+
|     Electronics|        101|               550|           1|
|         Fashion|        102|               180|           1|
|       Groceries|        103|                70|           1|
|          Health|        104|               120|           1|
+----------------+-----------+------------------+------------+



In [20]:
# data export 
customer_spendDF = t2DF.select("customer_id", "customer_name", "total_spend", "days_since_first_order")
category_summary_DF = t3DF.select("product_category", "total_order_amount" )

customer_spendDF.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("output_data/customer_spend")
category_summary_DF.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("output_data/category_summary")